In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import seaborn as sns

In [2]:
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

In [3]:
df = pd.read_csv('After_EDA.csv')
df.drop('Unnamed: 0', axis = 1, inplace= True)

In [4]:
df.head()

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,label
0,0,39.25,0.43,22.75,0.43,33.75,1.3,bending1
1,250,39.25,0.43,23.00,0.00,33.00,0.0,bending1
2,500,39.25,0.43,23.25,0.43,33.00,0.0,bending1
3,750,39.50,0.50,23.00,0.71,33.00,0.0,bending1
4,1000,39.50,0.50,24.00,0.00,33.00,0.0,bending1


In [5]:
X = df.drop('label', axis =1)
y = df['label']

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
scalar = StandardScaler()
X_scaled = scalar.fit_transform(X)

In [8]:
vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif["Features"] = X.columns

#let's check the values
vif

,vif,Features
0,1.000164,time
1,1.167131,avg_rss12
2,1.866994,var_rss12
3,1.327761,avg_rss13
4,1.780386,var_rss13
5,1.292629,avg_rss23
6,1.806817,var_rss23


In [9]:
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y, test_size= 0.25, random_state = 355)

## LogisticRegression

In [10]:
log_reg = LogisticRegression(multi_class='ovr')

log_reg.fit(x_train,y_train)

LogisticRegression(multi_class='ovr')

In [11]:
y_pred = log_reg.predict(x_test)

In [12]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.6404392504533548

In [13]:
# Confusion Matrix
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

array([[ 582,    3,   11,    0,   97,   40,    0],
       [  38,  464,   20,   97,   38,    9,    8],
       [   5,   19,  881,    5,   95,   35,  524],
       [   3,   58,    3, 1567,    9,  135,    3],
       [  60,  111,   66,  322,  595,  630,    5],
       [   6,    1,   34,  209,  343, 1238,    7],
       [   1,   31,  471,    1,    8,    8, 1030]], dtype=int64)

## DecisionTreeClassifier

In [17]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree 
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

In [18]:
#let's first visualize the tree on the data without doing any pre processing
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [23]:
y_pred = clf.predict(x_test)

In [26]:
clf.score(x_test, y_test)

0.7821881926254282

In [27]:
# Confusion Matrix
conf_mat = confusion_matrix(y_test,y_pred)
conf_mat

array([[ 647,   17,   13,    0,   43,   12,    1],
       [   8,  596,   21,   11,   23,    3,   12],
       [   9,   31, 1003,    3,   44,   49,  425],
       [   0,    1,    1, 1741,   11,   21,    3],
       [  44,   31,   52,   26, 1338,  284,   14],
       [  15,    5,   33,   28,  293, 1450,   14],
       [   1,   21,  503,    4,   21,   11,  989]], dtype=int64)

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,32,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'splitter' : ['best', 'random']
    
}

In [32]:
grid_search = GridSearchCV(estimator=clf,
                     param_grid=grid_param,
                     cv=5,
                    n_jobs =-1)

In [33]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']})

In [34]:
best_parameters = grid_search.best_params_
print(best_parameters)

{'criterion': 'gini', 'max_depth': 21, 'min_samples_leaf': 5, 'min_samples_split': 8, 'splitter': 'best'}


In [35]:
grid_search.best_score_

0.7793263388554973

In [36]:
clf = DecisionTreeClassifier(criterion = 'gini', max_depth =21, min_samples_leaf= 5, min_samples_split= 8, splitter ='best')
clf.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=21, min_samples_leaf=5, min_samples_split=8)

In [37]:
clf.score(x_test,y_test)

0.7790650816038687

## k-Nearest Neighbors

In [39]:
from sklearn.neighbors import KNeighborsClassifier

In [40]:
# let's fit the data into kNN model and see how well it performs:
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [41]:
y_pred = knn.predict(x_test)

In [42]:
knn.score(x_train,y_train)

0.8568358128757094

In [43]:
print("The accuracy score is : ", accuracy_score(y_test,y_pred))

The accuracy score is :  0.7867217408825308


In [44]:
param_grid = { 'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
               'leaf_size' : [18,20,25,27,30,32,34],
               'n_neighbors' : [3,5,7,9,10,11,12,13]
              }

In [45]:
gridsearch = GridSearchCV(knn, param_grid,verbose=3)

In [46]:
gridsearch.fit(x_train,y_train)

Fitting 5 folds for each of 168 candidates, totalling 840 fits
[CV 1/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3; total time=   0.8s
[CV 2/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3; total time=   0.7s
[CV 3/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3; total time=   0.7s
[CV 4/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3; total time=   0.7s
[CV 5/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3; total time=   0.7s
[CV 1/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=5; total time=   0.8s
[CV 2/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=5; total time=   0.7s
[CV 3/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=5; total time=   0.7s
[CV 4/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=5; total time=   0.7s
[CV 5/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=5; total time=   0.8s
[CV 1/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=7; total time=   0.8s
[CV 2/5] END algorithm=ball_tree, l

[CV 5/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=9; total time=   0.9s
[CV 1/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=10; total time=   1.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=10; total time=   1.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=10; total time=   1.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=10; total time=   0.9s
[CV 5/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=10; total time=   0.9s
[CV 1/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=11; total time=   0.9s
[CV 2/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=11; total time=   0.9s
[CV 3/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=11; total time=   1.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=11; total time=   1.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=11; total time=   1.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=25, n_neighbors=12; total time=   1.0s
[CV 2

[CV 5/5] END algorithm=ball_tree, leaf_size=30, n_neighbors=13; total time=   1.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=3; total time=   0.8s
[CV 2/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=3; total time=   0.8s
[CV 3/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=3; total time=   0.8s
[CV 4/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=3; total time=   0.8s
[CV 5/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=3; total time=   0.8s
[CV 1/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=5; total time=   0.9s
[CV 2/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=5; total time=   0.9s
[CV 3/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=5; total time=   0.9s
[CV 4/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=5; total time=   0.9s
[CV 5/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=5; total time=   0.8s
[CV 1/5] END algorithm=ball_tree, leaf_size=32, n_neighbors=7; total time=   0.9s
[CV 2/5] END al

[CV 5/5] END .algorithm=kd_tree, leaf_size=18, n_neighbors=9; total time=   0.4s
[CV 1/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=10; total time=   0.4s
[CV 2/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=10; total time=   0.4s
[CV 3/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=10; total time=   0.4s
[CV 4/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=10; total time=   0.4s
[CV 5/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=10; total time=   0.4s
[CV 1/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=11; total time=   0.4s
[CV 2/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=11; total time=   0.4s
[CV 3/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=11; total time=   0.4s
[CV 4/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=11; total time=   0.4s
[CV 5/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=11; total time=   0.4s
[CV 1/5] END algorithm=kd_tree, leaf_size=18, n_neighbors=12; total time=   0.4s
[CV 2/5] END algorithm=kd_tr

[CV 2/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=3; total time=   0.2s
[CV 3/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=3; total time=   0.2s
[CV 4/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=3; total time=   0.2s
[CV 5/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=3; total time=   0.2s
[CV 1/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=5; total time=   0.3s
[CV 2/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=5; total time=   0.3s
[CV 3/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=5; total time=   0.3s
[CV 4/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=5; total time=   0.3s
[CV 5/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=5; total time=   0.3s
[CV 1/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=7; total time=   0.3s
[CV 2/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=7; total time=   0.3s
[CV 3/5] END .algorithm=kd_tree, leaf_size=27, n_neighbors=7; total time=   0.3s
[CV 4/5] END .algorithm=kd_t

[CV 4/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=10; total time=   0.3s
[CV 5/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=10; total time=   0.3s
[CV 1/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=11; total time=   0.3s
[CV 2/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=11; total time=   0.3s
[CV 3/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=11; total time=   0.3s
[CV 4/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=11; total time=   0.3s
[CV 5/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=11; total time=   0.3s
[CV 1/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=12; total time=   0.3s
[CV 2/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=12; total time=   0.3s
[CV 3/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=12; total time=   0.3s
[CV 4/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=12; total time=   0.3s
[CV 5/5] END algorithm=kd_tree, leaf_size=32, n_neighbors=12; total time=   0.3s
[CV 1/5] END algorithm=kd_tr

[CV 1/5] END ...algorithm=brute, leaf_size=20, n_neighbors=5; total time=   2.6s
[CV 2/5] END ...algorithm=brute, leaf_size=20, n_neighbors=5; total time=   2.9s
[CV 3/5] END ...algorithm=brute, leaf_size=20, n_neighbors=5; total time=   3.2s
[CV 4/5] END ...algorithm=brute, leaf_size=20, n_neighbors=5; total time=   3.0s
[CV 5/5] END ...algorithm=brute, leaf_size=20, n_neighbors=5; total time=   2.6s
[CV 1/5] END ...algorithm=brute, leaf_size=20, n_neighbors=7; total time=   2.8s
[CV 2/5] END ...algorithm=brute, leaf_size=20, n_neighbors=7; total time=   2.9s
[CV 3/5] END ...algorithm=brute, leaf_size=20, n_neighbors=7; total time=   3.0s
[CV 4/5] END ...algorithm=brute, leaf_size=20, n_neighbors=7; total time=   2.9s
[CV 5/5] END ...algorithm=brute, leaf_size=20, n_neighbors=7; total time=   2.9s
[CV 1/5] END ...algorithm=brute, leaf_size=20, n_neighbors=9; total time=   2.8s
[CV 2/5] END ...algorithm=brute, leaf_size=20, n_neighbors=9; total time=   2.9s
[CV 3/5] END ...algorithm=br

[CV 3/5] END ..algorithm=brute, leaf_size=27, n_neighbors=11; total time=   3.0s
[CV 4/5] END ..algorithm=brute, leaf_size=27, n_neighbors=11; total time=   2.9s
[CV 5/5] END ..algorithm=brute, leaf_size=27, n_neighbors=11; total time=   2.8s
[CV 1/5] END ..algorithm=brute, leaf_size=27, n_neighbors=12; total time=   2.8s
[CV 2/5] END ..algorithm=brute, leaf_size=27, n_neighbors=12; total time=   2.9s
[CV 3/5] END ..algorithm=brute, leaf_size=27, n_neighbors=12; total time=   3.0s
[CV 4/5] END ..algorithm=brute, leaf_size=27, n_neighbors=12; total time=   2.9s
[CV 5/5] END ..algorithm=brute, leaf_size=27, n_neighbors=12; total time=   2.8s
[CV 1/5] END ..algorithm=brute, leaf_size=27, n_neighbors=13; total time=   2.8s
[CV 2/5] END ..algorithm=brute, leaf_size=27, n_neighbors=13; total time=   2.9s
[CV 3/5] END ..algorithm=brute, leaf_size=27, n_neighbors=13; total time=   3.1s
[CV 4/5] END ..algorithm=brute, leaf_size=27, n_neighbors=13; total time=   2.9s
[CV 5/5] END ..algorithm=bru

[CV 5/5] END ...algorithm=brute, leaf_size=34, n_neighbors=5; total time=   2.8s
[CV 1/5] END ...algorithm=brute, leaf_size=34, n_neighbors=7; total time=   2.9s
[CV 2/5] END ...algorithm=brute, leaf_size=34, n_neighbors=7; total time=   2.9s
[CV 3/5] END ...algorithm=brute, leaf_size=34, n_neighbors=7; total time=   3.0s
[CV 4/5] END ...algorithm=brute, leaf_size=34, n_neighbors=7; total time=   2.9s
[CV 5/5] END ...algorithm=brute, leaf_size=34, n_neighbors=7; total time=   2.8s
[CV 1/5] END ...algorithm=brute, leaf_size=34, n_neighbors=9; total time=   2.8s
[CV 2/5] END ...algorithm=brute, leaf_size=34, n_neighbors=9; total time=   2.9s
[CV 3/5] END ...algorithm=brute, leaf_size=34, n_neighbors=9; total time=   3.0s
[CV 4/5] END ...algorithm=brute, leaf_size=34, n_neighbors=9; total time=   2.9s
[CV 5/5] END ...algorithm=brute, leaf_size=34, n_neighbors=9; total time=   2.8s
[CV 1/5] END ..algorithm=brute, leaf_size=34, n_neighbors=10; total time=   3.0s
[CV 2/5] END ..algorithm=bru

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [18, 20, 25, 27, 30, 32, 34],
                         'n_neighbors': [3, 5, 7, 9, 10, 11, 12, 13]},
             verbose=3)

In [47]:
# let's see the  best parameters according to gridsearch
gridsearch.best_params_


{'algorithm': 'ball_tree', 'leaf_size': 18, 'n_neighbors': 7}

In [48]:
# we will use the best parameters in our k-NN algorithm and check if accuracy is increasing.
knn = KNeighborsClassifier(algorithm = 'ball_tree', leaf_size =18, n_neighbors =7)

In [49]:
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=18, n_neighbors=7)

In [50]:
knn.score(x_train,y_train)

0.8431675454209625

In [51]:
knn.score(x_test,y_test)

0.7876284505339513

In [64]:
y_pred = knn.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.7876284505339513

## RandomForestClassifier

In [52]:
from sklearn.ensemble import RandomForestClassifier

In [53]:
rand_clf = RandomForestClassifier(random_state=6)

In [54]:
rand_clf.fit(x_train,y_train)

RandomForestClassifier(random_state=6)

In [55]:
rand_clf.score(x_test,y_test)

0.8246020552085432

In [62]:
# cheking initial test accuracy
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.8268184565786822

## XGBClassifier

In [65]:
from xgboost import XGBClassifier

In [66]:
# fit model no training data
model = XGBClassifier()
model.fit(x_train, y_train)

C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:02:30] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [67]:
model.score(x_test,y_test)

0.8268184565786822

In [68]:
# cheking initial test accuracy
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.8268184565786822

In [69]:
param_grid={
   
    ' learning_rate':[1,0.5,0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
    
}

In [70]:
grid= GridSearchCV(XGBClassifier(),param_grid, verbose=3)

In [72]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[14:04:17] WARNING: ..\src\learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:04:17] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5] END . learning_rate=1, max_depth=3, n_estimators=10; total time=   0.1s


C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:04:17] WARNING: ..\src\learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:04:17] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5] END . learning_rate=1, max_depth=3, n_estimators=10; total time=   0.1s


C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:04:17] WARNING: ..\src\learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:04:17] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 3/5] END . learning_rate=1, max_depth=3, n_estimators=10; total time=   0.1s
[14:04:17] WARNING: ..\src\learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


...

C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 5/5] END  learning_rate=0.001, max_depth=20, n_estimators=200; total time=  13.4s


C:\Users\HP\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:34:42] WARNING: ..\src\learner.cc:541: 
Parameters: {  learning_rate } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:34:42] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameter

In [73]:
# To  find the parameters givingmaximum accuracy
grid.best_params_

{' learning_rate': 1, 'max_depth': 20, 'n_estimators': 100}

In [75]:
# Create new model using the same parameters
new_model=XGBClassifier(learning_rate= 1, max_depth= 20, n_estimators= 100)
new_model.fit(x_train, y_train)

[14:36:18] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [76]:
y_pred_new = new_model.predict(x_test)
accuracy_new = accuracy_score(y_test,y_pred_new)
accuracy_new

0.8336691517227484

### Save best model

In [78]:
import pickle

In [83]:
filename = 'model/xgboost_model.pickle'
pickle.dump(new_model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

In [84]:
# we'll save the scaler object as well for prediction
filename_scaler = 'model/scaler_model.pickle'
pickle.dump(scalar, open(filename_scaler, 'wb'))

scaler_model = pickle.load(open(filename_scaler, 'rb'))